In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from math import nan
import glob
import dask
import datetime
from dateutil.relativedelta import relativedelta
from functools import partial
import pandas as pd

dask.config.set(**{'array.slicing.split_large_chunks': True})

### Set up basic info

In [3]:
ystart = 1970 # start year of hindcasts 
yend = 2020 # end year of hindcasts
nyears = yend - ystart + 1
initmon=11
initmonstr = str(initmon).zfill(2)
nmems = 20 # the number of hindcast members
memstr = [str(i).zfill(3) for i in np.arange(1,nmems+1,1)] # generating member strings
#---!!!! ADD 100 for Nov initialization !!! I haven't done this yet
#memstr = [str(i+100).zfill(3) for i in np.arange(1,nmems+1,1)]

topdir = "/glade/campaign/cesm/development/espwg/SMYLE-CW3E-L83/timeseries/"
expname = "b.e21.BSMYLE-CW3E-L83.f09_g17."
outpath="/glade/campaign/cgd/cas/islas/python_savs/NCAR_CW3E_SMYLE/DATA_SORT/SST/"

### Set up the dask cluster

In [4]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client

cluster = PBSCluster(
    cores = 1,
    memory = '10GB',
    processes = 1,
    queue = 'casper',
    local_directory='$TMPDIR',
    resource_spec='select=1:ncpus=1:mem=10GB',
    project='P04010022',
    walltime='03:00:00',
    interface='mgt')

# scale up
cluster.scale(20)

# change your urls to the dask dashboard so that you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

/glade/u/home/islas/miniconda3/envs/islaenv24/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39317 instead
  f"Port {expected} is already in use.\n"


In [5]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/islas/proxy/39317/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/islas/proxy/39317/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.18.206.69:36721,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/islas/proxy/39317/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [7]:
cluster

PBSCluster(4ba99dc4, 'tcp://10.18.206.69:36721', workers=20, threads=20, memory=186.20 GiB)

### Set up the filelist

In [20]:
files = []
for iyear in np.arange(ystart,yend+1,1):
    filest = [glob.glob(topdir+expname+str(iyear)+'-'+initmonstr+'.'+str(imem).zfill(3)+"/atm/proc/tseries/month_1/*.TS.*.nc")[0] for imem in np.arange(1,20+1,1) ]
    files.append(filest)

In [22]:
def preprocessor(ds):
    # sort out the times so that each member has the time 
    timebndavg = np.array(ds.time_bnds, dtype='datetime64[s]').view('i8').mean(axis=1).astype('datetime64[s]')
    ds['time'] = timebndavg
    datestart = pd.to_datetime("1970-"+str(ds.isel(time=0).time.dt.month.values).zfill(2)+"-"+str(ds.isel(time=0).time.dt.day.values).zfill(2),
                               format="%Y-%m-%d")
    time = [ datestart + relativedelta(months = int(i)) for i in np.arange(0,ds.time.size,1) ] 
    ds['time'] = time
    return ds

In [23]:
dat = xr.open_mfdataset(files, combine='nested', concat_dim=['init_year', 'M'],
                        parallel=True, data_vars=['TS'], coords='minimal', compat='override', preprocess=partial(preprocessor))
dat['init_year'] = np.arange(ystart,yend+1,1)
dat = dat.TS
dat.load().to_netcdf(outpath+'SST_BSMYLE-CW3E-L83_mon_init'+initmonstr+'.nc')

In [24]:
cluster.close()

tornado.application - ERROR - Exception in callback <bound method Cluster._sync_cluster_info of PBSCluster(4ba99dc4, 'tcp://10.18.206.69:36721', workers=0, threads=0, memory=0 B)>
Traceback (most recent call last):
  File "/glade/u/home/islas/miniconda3/envs/islaenv24/lib/python3.7/site-packages/distributed/comm/tcp.py", line 205, in read
    frames_nbytes = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/glade/u/home/islas/miniconda3/envs/islaenv24/lib/python3.7/site-packages/distributed/core.py", line 787, in send_recv_from_rpc
    result = await send_recv(comm=comm, op=key, **kwargs)
  File "/glade/u/home/islas/miniconda3/envs/islaenv24/lib/python3.7/site-packages/distributed/core.py", line 640, in send_recv
    response = await comm.read(deserializers=deserializers)
  File "/glade/u/home/islas/miniconda3/envs/islaenv24/lib/python3